##Setup do ambiente

In [14]:
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd

project = 'fiery-webbing-421114' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

##Base de dados original

Consulta mostrando as 5 primeiras linhas da base de dados

In [15]:
sql = """
SELECT * FROM `bigquery-public-data.chicago_crime.crime`
"""
df = client.query(sql).to_dataframe()
df.head(5)

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,7061403,HR466721,2009-07-26 08:00:00+00:00,021XX N LAWLER AVE,2870,PUBLIC PEACE VIOLATION,PEEPING TOM,APARTMENT,False,False,...,31,19,24,1142377.0,1913686.0,2009,2018-02-28 03:56:25+00:00,41.919211,-87.752319,"(41.919210971, -87.752319001)"
1,1968351,HH157409,2002-01-27 06:45:00+00:00,054XX W DIVISION ST,5001,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,STREET,True,True,...,<NA>,<NA>,26,1139844.0,1907464.0,2002,2021-09-07 03:41:02+00:00,41.902184,-87.761778,"(41.902183806, -87.761778019)"
2,10517388,HZ259599,2016-05-11 12:12:00+00:00,049XX W HIRSCH ST,0325,ROBBERY,VEHICULAR HIJACKING,STREET,True,False,...,37,25,03,1143261.0,1908860.0,2016,2018-02-10 03:50:01+00:00,41.905951,-87.749192,"(41.905951428, -87.749191798)"
3,1825931,G655651,2001-10-31 02:45:00+00:00,014XX N CICERO AV,0920,MOTOR VEHICLE THEFT,ATT: AUTOMOBILE,PARKING LOT/GARAGE(NON.RESID.),False,False,...,<NA>,<NA>,07,1144079.0,1908911.0,2001,2015-08-17 03:03:40+00:00,41.906076,-87.746186,"(41.906076053, -87.746185669)"
4,9777651,HX427979,2014-09-14 11:45:00+00:00,016XX N MASON AVE,2092,NARCOTICS,SOLICIT NARCOTICS ON PUBLICWAY,SIDEWALK,True,False,...,29,25,26,1136502.0,1910246.0,2014,2018-02-10 03:50:01+00:00,41.909878,-87.773987,"(41.909878399, -87.773987238)"


##1ª QUERY
Porcentagem de crimes que tiveram prisão sobre o numero total de crimes no ano

In [17]:
sql = """
WITH total_crime_count AS
(SELECT
  year, COUNT(unique_key) AS contagem_total
FROM
  `bigquery-public-data.chicago_crime.crime`
GROUP BY
  year
),
n_crime_arrest AS
(SELECT
  year,
  COUNT(unique_key) AS contagem
FROM
  `bigquery-public-data.chicago_crime.crime`
GROUP BY
  year, arrest
HAVING
  arrest = true
)

SELECT
  nca.year,
  ROUND(nca.contagem/tcc.contagem_total*100,2) as perc_arrests
FROM
  total_crime_count AS tcc
LEFT JOIN
  n_crime_arrest AS nca
ON
  tcc.year = nca.year
ORDER BY year ASC;
"""
df_arrests = client.query(sql).to_dataframe()

df_arrests.head(24)

,year,perc_arrests
0,2001,29.21
1,2002,29.08
2,2003,29.75
3,2004,30.83
4,2005,31.06
5,2006,30.22
6,2007,30.17
7,2008,25.76
8,2009,28.21
9,2010,27.14


##2ª QUERY
Número de crimes do tipo "ROBBERY" ao longo dos anos e sua variação anual

In [19]:
sql = """
WITH robbery_n AS
(SELECT
  year, primary_type, COUNTIF(primary_type = "ROBBERY") AS n_crime
FROM
  `bigquery-public-data.chicago_crime.crime`
GROUP BY
  year, primary_type
HAVING
  primary_type = "ROBBERY"
ORDER BY
  year ASC
)

SELECT
  year,
  primary_type,
  n_crime,
CASE
  WHEN LAG(n_crime) OVER (ORDER BY year ASC) IS NULL THEN 0
  ELSE ROUND((n_crime - LAG(n_crime) OVER (ORDER BY YEAR ASC))/LAG(n_crime) OVER (ORDER BY YEAR ASC) * 100,2)
END AS
  variacao_anual
FROM
  robbery_n
ORDER BY
  year ASC;
"""

df_robbery = client.query(sql).to_dataframe()

df_robbery.head(24)

,year,primary_type,n_crime,variacao_anual
0,2001,ROBBERY,18441,0.00
1,2002,ROBBERY,18523,0.44
2,2003,ROBBERY,17332,-6.43
3,2004,ROBBERY,15978,-7.81
4,2005,ROBBERY,16047,0.43
5,2006,ROBBERY,15969,-0.49
6,2007,ROBBERY,15450,-3.25
7,2008,ROBBERY,16703,8.11
8,2009,ROBBERY,15981,-4.32
9,2010,ROBBERY,14275,-10.68


##3ª QUERY
Consulta que traz o tipo do crime e local do crime que mais ocorreu anualmente.
Ambos trazem a contagem de quantos crimes correspondentes foram registrados e também qual a porcentagem o tipo ou local representam da totalidade de crimes registrados.

In [20]:
sql = """
WITH type_crime_rank AS
(SELECT
  year, primary_type, COUNT(unique_key) AS contagem_tipo, ROW_NUMBER() OVER (PARTITION BY year ORDER BY COUNT(unique_key) DESC) as ranking_tipo
FROM
  `bigquery-public-data.chicago_crime.crime`
GROUP BY
  year, primary_type
),
local_crime_rank AS
(SELECT
  year, location_description, COUNT(unique_key) AS contagem_local, ROW_NUMBER() OVER (PARTITION BY year ORDER BY COUNT(unique_key) DESC) as ranking_local
FROM
  `bigquery-public-data.chicago_crime.crime`
GROUP BY
  year, location_description
),
total_crime_count AS
(SELECT
  year, COUNT(unique_key) AS contagem_total
FROM `bigquery-public-data.chicago_crime.crime`
GROUP BY
  year
)

SELECT
  tcr.year,
  tcr.primary_type,
  tcr.contagem_tipo,
  ROUND(tcr.contagem_tipo/tcc.contagem_total*100 ,2) AS percentual_tipo,
  lcr.location_description,
  lcr.contagem_local,
  ROUND(lcr.contagem_local/tcc.contagem_total*100, 2) AS percentual_local
FROM
  type_crime_rank as tcr
LEFT JOIN
  local_crime_rank as lcr
ON
  tcr.year = lcr.year
JOIN
  total_crime_count as tcc
ON
  tcr.year = tcc.year
WHERE
  tcr.ranking_tipo = 1
AND
  lcr.ranking_local = 1
ORDER BY
 year ASC;
 """

df_type_local_crime_rank = client.query(sql).to_dataframe()

df_type_local_crime_rank.head(24)

,year,primary_type,contagem_tipo,percentual_tipo,location_description,contagem_local,percentual_local
0,2001,THEFT,99280,20.43,STREET,150471,30.97
1,2002,THEFT,98334,20.20,STREET,147960,30.39
2,2003,THEFT,98876,20.77,STREET,145650,30.60
3,2004,THEFT,95464,20.34,STREET,138381,29.48
4,2005,THEFT,85684,18.88,STREET,123676,27.25
5,2006,THEFT,86241,19.24,STREET,118761,26.50
6,2007,THEFT,85156,19.48,STREET,117778,26.95
7,2008,THEFT,88437,20.70,STREET,112949,26.44
8,2009,THEFT,80975,20.61,STREET,98586,25.10
9,2010,THEFT,76758,20.72,STREET,93051,25.11
